In [1]:
import os, sys
sys.path = list(set([
    "../../lib/",
] + sys.path))
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Iterable, Any
from plotly import graph_objects as go, subplots as sp
from concurrent.futures import ProcessPoolExecutor as Exe
import networkx as nx
# import numba

from local.caching import load, save, save_exists
from local.figures import layout, xaxis_desc, yaxis_desc
# from txyl_common.biocyc_facade.pgdb import Pgdb, Dat, Traceable

In [74]:
def _circulant(n, k):
    return nx.Graph(nx.circulant_graph(n, [i+3 for i in range(k)]))

def _scale_free(n, k, seed:int=0, g=None):
    return nx.Graph(nx.barabasi_albert_graph(n, k, seed=seed, initial_graph=g))

# G = _circulant(100, 2)
# G = _scale_free(n, 99, _circulant(n, 1))
# ns, k = [15, 45, 45], 4
ns, k = [15, 75], 4
nlinks = 1
seed = 125
np.random.seed(seed)
G = nx.Graph()
for i, n in enumerate(ns):
    module = _scale_free(n, k, seed=seed+len(G))
    s = len(G)
    if s != 0:
        for _ in range(nlinks):
            a = np.random.randint(0, s)
            b = np.random.randint(s, s+len(module))
            G.add_edge(a, b)

    G.add_edges_from([(s+a, s+b) for a, b in module.edges])
    
pos = nx.kamada_kawai_layout(G)
# pos = nx.spring_layout(G)

In [88]:
# change in Q for each v in cluster if moved to other
def DeltaQ(G: nx.Graph, cluster, other):
    degrees_within = [G.degree[u] for u in cluster]
    degrees_out = [G.degree[u] for u in other]
    sum_in = np.sum(degrees_within)
    sum_out = np.sum(degrees_out)

    for v in cluster:
        neighbours = [u for u in G.neighbors(v)]
        edges_within = [u for u in neighbours if u in cluster]
        edges_out = [u for u in neighbours if u not in cluster]

        vsum_in = (sum_in - G.degree[v])
        vsum_out = sum_out
        Q = (len(edges_out) - len(edges_within)) / len(G.edges)
        Q -= G.degree[v] * (vsum_out - vsum_in) / (2 * len(G.edges)**2)
        
        # if Q > 0:
        #     yield Q, v
        yield Q, v

def ToSwap(G: nx.Graph, cluster: set[int], other: set[int]):
    to_give = DeltaQ(G, cluster, other) if len(cluster)>1 else []
    to_get = []

    if len(cluster) == len(G):
        to_give = [(q, v) for q, v in sorted(to_give, key=lambda t: t[0], reverse=True)]
    else:
        to_give = [(q, v) for q, v in sorted(to_give, key=lambda t: t[0], reverse=True) if q > 0]

    return [v for q, v in to_give][:1], [v for q, v in to_get][:1]
    # return [v for q, v in to_give], [v for q, v in to_get]

# a = set([77, 1, 55])
# a = set([77])
a = set(range(len(G)))
# a = set(range(1))

b = set(x for x in range(0, len(G)) if x not in a)

iterations = [(a, b)]
for i in range(30):
    give, get = (set(x) for x in ToSwap(G, a, b))
    newa = a.difference(give).union(get).copy()
    newb = b.difference(get).union(give).copy()
    a = newa
    b = newb
    iterations.append((a, b))
    # break
    
# print([x for x in give])
# print([x for x in get])
# print(a)
# print(b)

In [89]:
_layout: Any = layout.copy()
no_col = 'rgba(0, 0, 0, 0)'
axis_desc: dict = dict(
    linecolor=no_col, gridcolor=no_col, zerolinecolor=no_col, zerolinewidth=0,
    visible=False, range=(-1.1, 1.1), autorange=False,
)

_layout.update(dict(
    width=700, height=700,
    xaxis=axis_desc, yaxis=axis_desc,
))

def _plot(G: nx.Graph, pos: dict, groups: Iterable[set]):
    colors = "#0093CD, #FF9447, #90EA85".split(", ")
    glookup = {}
    for i, g in enumerate(groups):
        for v in g:
            glookup[v] = i
    cs = [colors[glookup[v]] for v in G.nodes]
    n_plot = go.Scatter(
            x=[x for i, (x, y) in enumerate(pos.values())],
            y=[y for i, (x, y) in enumerate(pos.values())],
            mode="markers", showlegend=False,
            marker=dict(
                color=cs,
            )
        )

    edges = []
    for a, b in G.edges:
        edges += [(pos[a]), (pos[b]), (None, None)]
    e_plot = go.Scatter(
        x=[x for x, y in edges], y=[y for x, y in edges],
        mode="lines", showlegend=False,
        line=dict(width=1, color="rgba(0, 0, 0, 0.2)"),
    )
    plots = (e_plot, n_plot)
    return plots

frames = []
sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Iteration:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 0, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}
for i, clusters in enumerate(iterations):
    slider_step = {
        "args": [
            [i],
            {"frame": {"duration": 100, "redraw": False},
            "mode": "immediate",
            "transition": {"duration": 0}}
        ],
        "label": i,
        "method": "animate"
    }
    sliders_dict["steps"].append(slider_step)
    frames.append(go.Frame(data=_plot(G, pos, clusters), name=str(i)))

fig = go.Figure(
    data=_plot(G, pos, iterations[0]),
    layout=go.Layout(
        updatemenus=[
            {
                "buttons": [
                    {
                        "args": [None, {"frame": {"duration": 100, "redraw": False},
                                        "fromcurrent": True, "transition": {"duration": 100,
                                                                            "easing": "quadratic-in-out"}}],
                        "label": "Play",
                        "method": "animate"
                    },
                    {
                        "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                        "mode": "immediate",
                                        "transition": {"duration": 0}}],
                        "label": "Pause",
                        "method": "animate"
                    }
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 87},
                "showactive": False,
                "type": "buttons",
                "x": 0.1,
                "xanchor": "right",
                "y": 0,
                "yanchor": "top"
            }
        ],
        sliders = [sliders_dict],
        **_layout
    ),
    frames=frames,
)

fig.show()

In [63]:
# pgdb_dir = Path("../../data/txyl_local/biocyc/pgdbs")
# np.random.seed(84)
# pgdb_files = list(os.listdir(pgdb_dir))
# sample_file = pgdb_files[np.random.randint(0, len(pgdb_files))]
# pgdb = Pgdb(pgdb_dir.joinpath(sample_file))
# _info = pgdb.GetInfo()
# for k in "PGDB-NAME, ORGANISM, NCBI-TAXONOMY-ID, PGDB-TIER, biocyc_facade_ver, Total_entries".split(", "):
#     v = _info[k].replace('"', "")
#     print(f"{k}:{' '*(25-len(k))}{v}")

In [64]:
# class MNetwork:
#     def __init__(self, reactions: dict) -> None:
#         ccode = {}
#         def encode(c):
#             if c not in ccode: 
#                 ccode[c] = len(ccode)
#             return ccode[c]

#         clookup = {}
#         rlookup = {}
#         for k, v in reactions.items():
#             lefts = v.get("LEFT", [])
#             rights = v.get("RIGHT", [])

#             cpds = set(lefts).union(rights)
#             rlookup[k] = {encode(c) for c in cpds}
#             for c in cpds:
#                 ci = encode(c)
#                 ref = clookup.get(ci, set())
#                 ref.add(k)
#                 clookup[ci] = ref

#         G = nx.Graph()
#         for v, rxns in clookup.items():
#             for rxn in rxns:
#                 for u in rlookup[rxn]:
#                     G.add_edge(u, v)

#         self.G = G
#         self._ccode = ccode
#         revcode = ['']*len(ccode)
#         for k, i in ccode.items():
#             revcode[i] = k
#         self._revcode = revcode

#     def Decode(self, index: int):
#         return self._revcode[index] if index < len(self._revcode) else None
    
#     def Encode(self, cpd: str):
#         return self._ccode.get(cpd)
    
# net = MNetwork(pgdb.GetDataTable(Dat.REACTIONS))

In [65]:
# len(net.G.edges), len(net.G)

In [66]:
# nx.cluster.average_clustering(net.G)

In [67]:
# ds = sorted(dict(net.G.degree).items(), key=lambda t: t[1], reverse=True)
# [(net.Decode(i), d) for i, d in ds][:5]

In [68]:
from networkx import community as nx_com

def BinaryModularity(G: nx.Graph, cluster: Iterable[int]):
    cluster_s = set(cluster)
    other_s = {n for n in G.nodes if n not in cluster_s}

    ak_c = np.sum([G.degree[n] for n in cluster]) / (2*len(G.edges))
    ak_o = np.sum([G.degree[n] for n in other_s]) / (2*len(G.edges))

    def _edges_within(group):
        edges_within = 0
        for a, b in G.edges:
            if a in group and b in group: edges_within+=1
        return edges_within

    e_c = _edges_within(cluster_s)/len(G.edges)
    e_o = _edges_within(other_s)/len(G.edges)

    Q = (e_c - ak_c**2) + (e_o - ak_o**2)
    return Q

size = 1000
ea = [(0, i+1) for i in range(size-1)]
eb = [(size, size+i+1) for i in range(size-1)]
testG = nx.Graph()
testG.add_edges_from(ea)
testG.add_edges_from(eb)
testG.add_edge(0, size)

BinaryModularity(testG, range(0, 2*size, 2))
BinaryModularity(testG, range(size-1))

0.49899937462478106

In [69]:
size = 10
ea = [(0, i+1) for i in range(size-1)]
ea += [(1, i+1) for i in range(size-1)]
eb = [(size, size+i+1) for i in range(size-1)]
eb += [(size+1, size+i+1) for i in range(size-1)]
testG = nx.Graph()
testG.add_edges_from(ea)
testG.add_edges_from(eb)
testG.add_edge(0, size)

G = testG
# cluster = range(size-1)
cluster = range(0, 2*size, 2)

cluster = set(cluster)
other_s = {n for n in G.nodes if n not in cluster}

def _to_swap(cluster):
    for v in cluster:
        neighbours = [u for u in G.neighbors(v)]
        edges_within = len([u for u in neighbours if u in cluster])
        if edges_within < len(neighbours)/2:
            yield v

to_give = _to_swap(cluster)
to_get = _to_swap(other_s)

list(to_give)

[]